# This notebook uses Papermill to batch run models

In [1]:
%load_ext lab_black
import papermill as pm
import os
from multiprocessing import Pool

## Batch configurations

In [2]:
batch_name = "O2P_main"

# Create batch cfgs
batch_cfgs = []
i = 0

for noise in [0., 1., 2., 4., 8.]:
    for h in [25, 50, 100, 250]:
        for c_unit in [25, 50]:
            if c_unit == 25:
                for c_stage in [0, 195, 300]:
                    # Clean up unit = 25
                    i += 1
                    code_name = batch_name + "_model_{:04d}".format(i)
                    batch_cfg = dict(
                        sn=i,
                        in_notebook="basicOSP_master.ipynb",
                        code_name=code_name,
                        model_folder="models/" + code_name + "/",
                        out_notebook="models/" + code_name + "/output.ipynb",
                        params=dict(
                            code_name=code_name,
                            sample_name='hs04',
                            sample_rng_seed=329,
                            tf_rng_seed=123,
                            use_semantic=False,
                            sem_param_gf=0.,
                            sem_param_gi=0.,
                            sem_param_kf=0.,
                            sem_param_ki=0.,
                            sem_param_hf=0.,
                            sem_param_hi=0.,
                            o_input_dim=119,
                            hidden_units=h,
                            pho_units=250,
                            cleanup_units=c_unit,
                            rnn_activation='sigmoid',
                            regularizer_const=5e-6,
                            embed_attractor_cfg=
                            'models/Attractor_{}/model_config.json'.
                            format(c_unit),
                            embed_attractor_h5='ep{0:04d}.h5'.format(c_stage),
                            p_noise=noise,  # i.e. w_pp, w_pc, and w_cp noise
                            tau=0.2,
                            max_unit_time=4.,
                            n_mil_sample=.04,
                            batch_size=128,
                            learning_rate=0.005,
                            save_freq=1,
                            bq_dataset=batch_name
                        )
                    )
                    batch_cfgs.append(batch_cfg)

            if c_unit == 50:
                for c_stage in [0, 70, 125]:
                    i += 1
                    code_name = batch_name + "_model_{:04d}".format(i)
                    batch_cfg = dict(
                        sn=i,
                        in_notebook="basicOSP_master.ipynb",
                        code_name=code_name,
                        model_folder="models/" + code_name + "/",
                        out_notebook="models/" + code_name + "/output.ipynb",
                        params=dict(
                            code_name=code_name,
                            sample_name='hs04',
                            sample_rng_seed=329,
                            tf_rng_seed=123,
                            use_semantic=False,
                            sem_param_gf=0.,
                            sem_param_gi=0.,
                            sem_param_kf=0.,
                            sem_param_ki=0.,
                            sem_param_hf=0.,
                            sem_param_hi=0.,
                            o_input_dim=119,
                            hidden_units=h,
                            pho_units=250,
                            cleanup_units=c_unit,
                            rnn_activation='sigmoid',
                            regularizer_const=5e-6,
                            embed_attractor_cfg=
                            'models/Attractor_{}/model_config.json'.
                            format(c_unit),
                            embed_attractor_h5='ep{0:04d}.h5'.format(c_stage),
                            p_noise=noise,  # i.e. w_pp, w_pc, and w_cp noise
                            tau=0.2,
                            max_unit_time=4.,
                            n_mil_sample=.04,
                            batch_size=128,
                            learning_rate=0.005,
                            save_freq=1,
                            bq_dataset=batch_name
                        )
                    )
                    batch_cfgs.append(batch_cfg)


# Run
def run_batch(cfg):
    try:
        print("Running model {}".format(cfg['sn']))

        if not os.path.exists(cfg['model_folder']):
            os.mkdir(cfg['model_folder'])

        pm.execute_notebook(
            cfg['in_notebook'],
            cfg['out_notebook'],
            parameters=cfg['params'],
        )

    except:
        print("Error occur in {}".format(cfg['code_name']))

## Run batch

In [ ]:
# Run in parallel pool
# with Pool(4) as pool:
#     pool.map(run_batch, batch_cfgs)

# Push results to BQ
from meta import model_cfg, connect_gbq
import pandas as pd
from tqdm import tqdm

# Make connection to bq
bq = connect_gbq()

for sn in tqdm(range(len(batch_cfgs) + 1)):

    model_folder = 'models/{0:s}_model_{1:04d}'.format(batch_name, sn + 1)

    # Load model config 
    cfg = model_cfg(None)
    cfg.load_cfg_json(model_folder + '/model_config.json')
    cfg.bq_dataset = batch_name

    # Load Strain and Grain
    strain_i_hist = pd.read_csv(model_folder + '/result_strain_item.csv')
    grain_i_hist = pd.read_csv(model_folder + '/result_grain_item.csv')

#     bq.push_all(cfg, strain_i_hist, grain_i_hist)

#### Shutdown compute engine

In [ ]:
from time import sleep
sleep(30)
!sudo poweroff  

## Compile results

In [3]:
import os, json
import pandas as pd
from meta import connect_gbq
from evaluate import vis
from tqdm import tqdm

conn = connect_gbq()
cfgs = conn.read_bq_cfg(batch_name)

# Read cfg files from BQ
print('===== Batch level hyperparams (columns that have >1 unique value) =====')
for i, x in enumerate(cfgs.columns):
    if not x == 'code_name':
        if not x == 'uuid':
            if len(cfgs[x].unique()) > 1:
                print(
                    'Column <{}> has these unique values: {}'.format(
                        x, cfgs[x].unique()
                    )
                )

# Parse each run by batch_eval, which aggregate item level data to condition level
# and merge Grain and Strain into one single file (Using local files instead of BQ,
# may use BQ for way way more data... >5Gbs I guess)

models_path = []
for i in range(len(cfgs)):
    models_path.append('models/' + batch_name + '_model_{0:04d}'.format(i + 1))

batch_acc = pd.DataFrame()

for i in tqdm(range(len(cfgs))):

    model_path = 'models/' + batch_name + '_model_{0:04d}'.format(i + 1)

    this_eval = vis(
        model_path, 'result_strain_item.csv', 'result_grain_item.csv'
    )  # Eval lesion and grain
    this_eval.parse_cond_df()
    batch_acc = pd.concat([batch_acc, this_eval.cdf], ignore_index=True)

df = pd.merge(batch_acc, cfgs, 'left', 'code_name')
print('Done')

  0%|          | 0/120 [00:00<?, ?it/s]

===== Batch level hyperparams (columns that have >1 unique value) =====
Column <hidden_units> has these unique values: [ 50 100 250  25]
Column <cleanup_units> has these unique values: [50 25]
Column <embed_attractor_cfg> has these unique values: ['models/Attractor_50/model_config.json'
 'models/Attractor_25/model_config.json']
Column <embed_attractor_h5> has these unique values: ['ep0070.h5' 'ep0195.h5' 'ep0300.h5' 'ep0000.h5' 'ep0125.h5']
Column <w_pp_noise> has these unique values: [8. 2. 4. 0. 1.]
Column <w_pc_noise> has these unique values: [8. 2. 4. 0. 1.]
Column <w_cp_noise> has these unique values: [8. 2. 4. 0. 1.]


100%|██████████| 120/120 [00:55<00:00,  2.16it/s]

Done


In [14]:
model_path = 'models/' + batch_name + '_model_{0:04d}'.format(10)

this_eval = vis(
    model_path, 'result_strain_item.csv', 'result_grain_item.csv'
)  # Eval lesion and grain
this_eval.parse_cond_df()

In [4]:
def cal_attractor_acc(cleanup_units, embed_attractor_h5):
    if cleanup_units == 25:
        if embed_attractor_h5 == 'ep0000.h5':
            acc = 0.
        if embed_attractor_h5 == 'ep0195.h5':
            acc = 0.6
        if embed_attractor_h5 == 'ep0300.h5':
            acc = 0.9

    if cleanup_units == 50:
        if embed_attractor_h5 == 'ep0000.h5':
            acc = 0.
        if embed_attractor_h5 == 'ep0070.h5':
            acc = 0.6
        if embed_attractor_h5 == 'ep0125.h5':
            acc = 0.9

    return acc

In [5]:
tmp_acc = []

for i in df.index:
    tmp_acc.append(
        cal_attractor_acc(df.cleanup_units[i], df.embed_attractor_h5[i])
    )

df['attactor_acc'] = tmp_acc

# Save to h5 format
df.to_hdf('batch_eval/{}_cdf.h5'.format(batch_name), key='df', mode='w')

### Read from file

In [27]:
df = pd.read_hdf('batch_eval/{}_cdf.h5'.format(batch_name), 'df')

In [30]:
df.columns

Index(['code_name', 'epoch', 'timestep', 'condition_pf', 'sample_mil',
       'unit_time', 'input_s', 'acc', 'sse', 'cond', 'exp', 'condition',
       'acc_small_grain', 'sse_small_grain', 'acc_large_grain',
       'sse_large_grain', 'uuid', 'sample_name', 'sample_rng_seed',
       'tf_rng_seed', 'use_semantic', 'sem_param_gf', 'sem_param_gi',
       'sem_param_kf', 'sem_param_ki', 'sem_param_hf', 'sem_param_hi',
       'o_input_dim', 'hidden_units', 'pho_units', 'cleanup_units',
       'embed_attractor_cfg', 'embed_attractor_h5', 'w_oh_noise', 'w_hp_noise',
       'w_pp_noise', 'w_pc_noise', 'w_cp_noise', 'tau', 'max_unit_time',
       'n_timesteps', 'n_mil_sample', 'nEpo', 'batch_size', 'steps_per_epoch',
       'rnn_activation', 'w_initializer', 'regularizer_const', 'learning_rate',
       'save_freq', 'save_freq_sample', 'eval_freq', 'bq_dataset',
       'attactor_acc'],
      dtype='object')

## Overview plot

In [11]:
df_ind = df.loc[(df.timestep == df.timestep.max()) &
                (df.cond.isin(['INC_LF', 'ambiguous', 'unambiguous'])), [
                    'code_name', 'epoch', 'hidden_units', 'cleanup_units',
                    'w_pp_noise', 'attactor_acc', 'acc', 'exp'
                ]]

In [12]:
df_ind.pivot_table(index=['code_name', 'epoch'], columns='exp')

acc         attactor_acc        cleanup_units  \
exp                           grain  strain        grain strain         grain   
code_name           epoch                                                       
O2P_main_model_0001 1      0.004167  0.0125          0.0    0.0            25   
                    2      0.095833  0.1000          0.0    0.0            25   
                    3      0.191667  0.1125          0.0    0.0            25   
                    4      0.254167  0.1250          0.0    0.0            25   
                    5      0.600000  0.3250          0.0    0.0            25   
...                             ...     ...          ...    ...           ...   
O2P_main_model_0120 60     0.000000  0.0000          0.9    0.9            50   
                    65     0.000000  0.0250          0.9    0.9            50   
                    70     0.000000  0.0250          0.9    0.9            50   
                    75     0.033333  0.0250          0.9    0.9            50   
                    80     0.033333  0.0500          0.9    0.9            50   

                                 hidden_units        w_pp_noise         
exp                       strain        grain strain      grain strain  
code_name           epoch                                               
O2P_main_model_0001 1         25           25     25        0.0    0.0  
                    2         25           25     25        0.0    0.0  
                    3         25           25     25        0.0    0.0  
                    4         25           25     25        0.0    0.0  
                    5         25           25     25        0.0    0.0  
...                          ...          ...    ...        ...    ...  
O2P_main_model_0120 60        50          250    250        8.0    8.0  
                    65        50          250    250        8.0    8.0  
                    70        50          250    250        8.0    8.0  
                    75        50          250    250        8.0    8.0  
                    80        50          250    250        8.0    8.0  

[2400 rows x 10 columns]

In [ ]:
# Plotting results
import altair as alt

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])
sel_cond = alt.selection(
    type="multi", on="click", fields=["cond"], bind="legend"
)

# Heatmap for final epoch & timestep (Overview)
plot_timestep = df.timestep.max()
# plot_timestep = 19

# Plot strain

df_ov_strain = df[(df.epoch == df.epoch.max()) &
                  (df.timestep == plot_timestep) & (df.exp == 'strain')]

overview_strain = (
    alt.Chart(df_ov_strain).mark_rect().encode(
        x="hidden_units:O",
        y="w_pp_noise:O",
        column='attactor_acc',
        row='cleanup_units',
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)

df_ov_grain = df[(df.epoch == df.epoch.max()) & (df.timestep == plot_timestep) &
                 (df.exp == 'grain')]

overview_grain = (
    alt.Chart(df_ov_grain).mark_rect().encode(
        x="hidden_units:O",
        y="w_pp_noise:O",
        column='attactor_acc',
        row='cleanup_units',
        color=alt.Color("acc", scale=alt.Scale(scheme="redyellowgreen")),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_run)
)



# Accuracy over epoch at last time step for selected model
last_time_point = df[df.timestep == df.timestep.max()]

acc_plot = (
    alt.Chart(last_time_point).mark_line().encode(
        y=alt.Y("acc:Q", scale=alt.Scale(domain=(0, 1))),
        x="epoch",
        color="cond",
        opacity=alt.condition(sel_cond, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "acc"],
    ).add_selection(sel_cond).transform_filter(sel_run).properties(
        title="Full model at final time step"
    )
)
overview = overview_strain & overview_grain
plot = overview | acc_plot
plot

### 

### Save plot

In [ ]:
plot.save('batch_eval/O2P_l2reg.html')

## Eval one model

In [ ]:
from meta import model_cfg
from evaluate import vis

code_name = '{0:}_model_{1:04d}'.format(batch_name, 47)

# Load cfg from json
cfg = model_cfg(None)
cfg.load_cfg_json('models/' + code_name + '/model_config.json')

vis = vis(
    cfg.path_model_folder, 'result_strain_item.csv', 'result_grain_item.csv'
)

vis.parse_cond_df()

full = vis.plot_dev_interactive('acc').properties(title='Full input')

full

### Flexible development plot

In [ ]:
vis.plot_dev('acc', exp=None, condition='cond', timestep=2)

### Flexible time plot

In [ ]:
vis.plot_time('acc', exp='strain', condition='cond', epoch=400)